this script makes a movie showing the cardiac function for each case. (3x3 SAX)

In [1]:
import sys
sys.path.append("../../../")
import CMR_HFpEF_Analysis.Defaults as Defaults
import CMR_HFpEF_Analysis.functions_collection as ff
import CMR_HFpEF_Analysis.Image_utils as util

import os
import numpy as np
import nibabel as nb
from scipy.ndimage import zoom
import SimpleITK as sitk
from PIL import Image
import cv2
import pandas as pd
import shutil

main_path = '/mnt/mount_zc_NAS/HFpEF/data/HFpEF_data/'

In [3]:
spreadsheet = pd.read_excel(os.path.join(main_path ,'Patient_list', 'Important_HFpEF_Patient_list_unique_patient_w_readmission_finalized.xlsx' ))
patient_list = spreadsheet.iloc[0:50]


In [6]:
for patient_index in range(0,patient_list.shape[0]):
    patient_id_num = patient_list['OurID'].iloc[patient_index]
    patient_id = ff.XX_to_ID_00XX(patient_id_num)
    patient_row = spreadsheet.loc[spreadsheet['OurID'] == patient_id_num]
    ED = patient_row['ED'].values[0].astype(int)
    ES = patient_row['ES'].values[0].astype(int)

    WL = 250
    WW = 250

    case_images = ff.sort_timeframe(ff.find_all_target_files(['*.nii.gz'],os.path.join(main_path, 'nii_img', patient_id)),2,'e')
    if ED != 1:
        case_images = np.concatenate((case_images[ED-1 : ] , case_images[:ED]))

    # load segmentation
    seg = nb.load(os.path.join(main_path,'nii_manual_seg', patient_id, 'SAX_ED_seg.nii.gz')).get_data()
    slice_nonzero = [i for i in range(seg.shape[2]) if np.sum(seg[:,:,i] >= 1)>0]
    
    # decide it starts from basal or apex
    start_slice = np.copy(seg[:,:,slice_nonzero[1]]); start_slice[start_slice >0] = 1
    end_slice = np.copy(seg[:,:,slice_nonzero[-1 - 2]]); end_slice[end_slice >0] = 1
    if np.sum(start_slice) < np.sum(end_slice):
        start_slice_name = 'apex'
    else:
        start_slice_name = 'base'


    # make images
    for n in range(0, case_images.shape[0]):
        img = nb.load(case_images[n]).get_fdata()
        img = np.flip(np.rollaxis(img,1,0), axis = 0)

        img = img[:,:,slice_nonzero]
        if start_slice_name == 'apex':
            img = img[:,:,::-1]

        h = 300;w = 300
        I = np.zeros((h*3,w*3,3))

        lv_slices = np.arange(0, img.shape[2], 1).tolist()
        if len(lv_slices) >= 9:
            movie_slices_index = np.linspace(0, len(lv_slices)-1 ,9); movie_slices_index = [int(ss) for ss in movie_slices_index]
            movie_slices = [lv_slices[ss] for ss in movie_slices_index]
        if len(lv_slices) < 9:
            movie_slices = lv_slices + [100] * (9 - len(lv_slices))
        if len(lv_slices) <3 :
            continue

        for row in range(0,3):
            for column in range(0,3):
                slice_index = movie_slices[(row+1) * (column + 1) - 1]
                if slice_index == 100:
                    sax = np.zeros([h, w])
                else:
                    sax = img[(img.shape[0]//2 -20 - 50) : (img.shape[0]//2 -20+ 50), (img.shape[1]//2  - 50) : (img.shape[1]//2  + 50), slice_index]
                    sax = zoom(sax, (3,3))
                    
                I[h*row : h*row + h , w*column : w*column + w, 0 ] = ff.set_window(sax, WL, WW)
                I[h*row : h*row + h , w*column : w*column + w, 1 ] = ff.set_window(sax, WL, WW)
                I[h*row : h*row + h , w*column : w*column + w, 2 ] = ff.set_window(sax, WL, WW)

        ff.make_folder([os.path.join(main_path, 'movies', patient_id), os.path.join(main_path, 'movies', patient_id, 'pngs')])
        Image.fromarray((I * 255).astype('uint8')).save(os.path.join(main_path, 'movies', patient_id, 'pngs', 'tf_' + str(n)) + '.png')

    pngs = ff.sort_timeframe(ff.find_all_target_files(['*'], os.path.join(main_path, 'movies', patient_id, 'pngs')),1,'_')
    ff.make_movies(os.path.join(main_path, 'movies', patient_id, patient_id + '_movie.mp4'),pngs,len(pngs) )

    shutil.rmtree(os.path.join(main_path, 'movies', patient_id, 'pngs'))
   

/tmp/ipykernel_534841/176856314.py:16: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  seg = nb.load(os.path.join(main_path,'nii_manual_seg', patient_id, 'SAX_ED_seg.nii.gz')).get_data()
/tmp/ipykernel_534841/176856314.py:16: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  seg = nb.load(os.path.join(main_path,'nii_manual_seg', patient_id, 'SAX_ED_seg.nii.gz')).get_data()
/tmp/ipykernel_534841/176856314.py:16: DeprecationWarning: get_data() is deprecated in favor of get_